# Inception-v3

In [ ]:
%conda install -y gdown

In [ ]:
import zipfile
from pathlib import Path
import datetime

from tensorflow.config import list_physical_devices
from tensorflow.keras.utils import plot_model # , load_img, img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

In [ ]:
zip_name = "dataset.zip"
wd = Path("/kaggle/working")
extract_path = Path(wd, "data")
model_path = Path(wd, "models/inception_v3.h5")
train_path = Path(extract_path, "train")
test_path = Path(extract_path, "test")

class_names = list()

## Model settings

In [ ]:
batch_size = 64
img_size = 299
epochs = 4
seed = 27
validation_split = 0.2

## Download dataset

In [ ]:
gdown.download(
    "https://drive.google.com/uc?id=1LOOzAQOIAXiWF7yjnGwaW3AQXKtvP1Eq",
    zip_name
)

zip_ref = zipfile.ZipFile(Path(wd, zip_name), 'r')
zip_ref.extractall(extract_path)
zip_ref.close()

## Get class names from the training directory

In [ ]:
class_names = [class_name.name for class_name in train_path.iterdir()]
for class_name in class_names:
    print(class_name)

## Check number of GPUs

In [ ]:
print("Num GPUs Available: {}".format(len(list_physical_devices('GPU'))))

## Load training and validation data

In [ ]:
training_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='training',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

validation_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='validation',
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

## Define model

In [ ]:
model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(img_size, img_size, 3),
    pooling='avg'
)

In [ ]:
top = Dense(units=2048, activation='relu')(model.output)
top = Dropout(rate = 0.2)(top)
top = Dense(units=len(class_names), activation='softmax')(top)

final_model = Model(
    inputs=model.input,
    outputs=top
)

optimizer = RMSprop(learning_rate=0.0001)

final_model.compile(
    optimizer=optimizer,
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
plot_model(
    final_model, 
    to_file='inception_v3.png', 
    show_shapes=True,
    show_layer_names=True
)

In [ ]:
history = final_model.fit(
    training_data,
    validation_data=validation_data,
    epochs=epochs,
    shuffle=True
)

In [ ]:
final_model.save(model_path)

In [ ]:
test_data = image_dataset_from_directory(
    directory=test_path,
    labels='inferred',
    class_names=class_names,
    label_mode='int',
    batch_size=batch_size,
    image_size=(img_size, img_size)
)

evaluation = final_model.evaluate(
    test_data,
    batch_size=batch_size
)
print("Loss is {:.4f}".format(evaluation[0]))
print("Accuracy is {:.2f}%".format(100 * evaluation[1]))